# 指派问题 例7 p. 126

例7 有一份中文说明书,需译成英、日、德、俄四种文字。分别记作E、J、G、R。现 有甲、乙、丙、丁四人。他们将中文说明书翻译成不同语种的说明书所需时间如表 5-7 所 示。问应指派何人去完成何工作 , 使所需总时间最少 ?

参考：`facility.py` http://www.gurobi.com/documentation/7.0/examples/facility_py.html

### Step 1: Import functions from the gurobipy module

In [1]:
from gurobipy import *


### Step 2: Create empty model

In [2]:
m = Model("facility")

# Warehouse demand in thousands of units
# 销量
# 最小销量
demand_min = [1, 1, 1, 1] # [0, 0, 0, 0]
# 最大销量
demand_max = [1, 1, 1, 1]
demand = demand_min  # 便于 len(demand) 取数组大小

# Plant capacity in thousands of units
# 产量
capacity = [1, 1, 1, 1]

# Fixed costs for each plant
# 无固定支出
fixedCosts = [0, 0, 0, 0]

# Transportation costs per thousand units
transCosts = [[2, 15, 13 , 4],
              [10, 4, 14 , 15],
              [9, 14, 16 , 13],
              [7, 8, 11 , 9],
             ]

transCostsDict = {}



### Step 3: Create activitiy variables

In [3]:
# Range of plants and warehouses
# 竖轴 提供方：人员
plants = range(len(capacity))
# 横轴 消费方：任务
warehouses = range(len(demand))

# 将cost矩阵转为字典，预备计算代价
for i in range(len(capacity)):
    for j in range(len(demand)):
        transCostsDict[(i,j)] = transCosts[i][j]
print "cost矩阵", transCosts
print "cost字典()", transCostsDict
##
# facility.py 中 考虑工厂开放存在cost，增加关闭变量
# Plant open decision variables: open[p] == 1 if plant p is open.
# open = m.addVars(plants,
#                  vtype=GRB.BINARY,
#                  obj=fixedCosts,
#                  name="open")
# 
## 我们这里假设所有工厂都在开放状态
open = [1] * len(plants)


# 修改为整形变量 
transport = m.addVars(plants, warehouses, vtype=GRB.BINARY, name="trans")
transport

cost矩阵 [[2, 15, 13, 4], [10, 4, 14, 15], [9, 14, 16, 13], [7, 8, 11, 9]]
cost字典() {(0, 1): 15, (1, 2): 14, (3, 2): 11, (0, 0): 2, (3, 3): 9, (3, 0): 7, (3, 1): 8, (2, 1): 14, (0, 2): 13, (2, 0): 9, (1, 3): 15, (2, 3): 13, (2, 2): 16, (1, 0): 10, (0, 3): 4, (1, 1): 4}


{(0, 0): <gurobi.Var trans[0,0]>,
 (0, 1): <gurobi.Var trans[0,1]>,
 (0, 2): <gurobi.Var trans[0,2]>,
 (0, 3): <gurobi.Var trans[0,3]>,
 (1, 0): <gurobi.Var trans[1,0]>,
 (1, 1): <gurobi.Var trans[1,1]>,
 (1, 2): <gurobi.Var trans[1,2]>,
 (1, 3): <gurobi.Var trans[1,3]>,
 (2, 0): <gurobi.Var trans[2,0]>,
 (2, 1): <gurobi.Var trans[2,1]>,
 (2, 2): <gurobi.Var trans[2,2]>,
 (2, 3): <gurobi.Var trans[2,3]>,
 (3, 0): <gurobi.Var trans[3,0]>,
 (3, 1): <gurobi.Var trans[3,1]>,
 (3, 2): <gurobi.Var trans[3,2]>,
 (3, 3): <gurobi.Var trans[3,3]>}

### Step 4: Set objective function

In [4]:
# 为何这里的 obj=transCosts ?
# transport = m.addVars(warehouses, plants, name="trans")
obj = transport.prod(transCostsDict)
m.setObjective( obj , GRB.MINIMIZE )
# m.modelSense = GRB.MINIMIZE
# m.setObjective(quicksum(transport.prod(transCosts)), GRB.MINIMIZE)
print "目标函数 obj:", obj

目标函数 obj: <gurobi.LinExpr: 2.0 trans[0,0] + 15.0 trans[0,1] + 13.0 trans[0,2] + 4.0 trans[0,3] + 10.0 trans[1,0] + 4.0 trans[1,1] + 14.0 trans[1,2] + 15.0 trans[1,3] + 9.0 trans[2,0] + 14.0 trans[2,1] + 16.0 trans[2,2] + 13.0 trans[2,3] + 7.0 trans[3,0] + 8.0 trans[3,1] + 11.0 trans[3,2] + 9.0 trans[3,3]>


### Step 5: Add constraints

In [5]:
# Production constraints
# Note that the right-hand limit sets the production to zero if the plant
# is closed
# 
# 关闭此约束条件时：  
#  甲（A0）：因为效率高，可以同时完成2个工作。 最优值为较少为： 21.0
#
# SOLUTION: 矩阵解为
#     [B0] [B1] [B2] [B3]
# [A0] 1.0 0.0 0.0 1.0
# [A1] 0.0 1.0 0.0 0.0
# [A2] 0.0 0.0 0.0 0.0
# [A3] 0.0 0.0 1.0 0.0
# 最优值为： 21.0
# 
# 打开此约束条件时：  甲（A0）：只能完成1个工作。 最优值为： 21.0
# 
m.addConstrs(
    (transport.sum(p, '*') == capacity[p] for p in plants),
    "Capacity")


# # Demand constraints
# m.addConstrs(
#     (transport.sum('*', w) <= demand_max[w] for w in warehouses),
#     "Demand")
#
# 
m.addConstrs(
    (transport.sum('*', w) == demand_min[w] for w in warehouses),
    "Demand")

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>}

### Step 6: Solve model

In [6]:
m.optimize()

Optimize a model with 8 rows, 16 columns and 32 nonzeros
Variable types: 0 continuous, 16 integer (16 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 49
Presolve time: 0.00s
Presolved: 8 rows, 16 columns, 32 nonzeros
Variable types: 0 continuous, 16 integer (16 binary)

Root relaxation: objective 2.800000e+01, 7 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      28.0000000   28.00000  0.00%     -    0s

Explored 0 nodes (7 simplex iterations) in 0.02 seconds
Thread count was 4 (of 4 available processors)

Solution count 2: 28 49 
Pool objective bound 28

Optimal solution found (tolerance 1.00e-04)
Best objective 2.800000000000e+01, best bound 2.800000000000e+01, gap 0.0000%


### Step 8: Print variable values for optimal solution

In [7]:
# Print solution
print('\nTOTAL COSTS: %g' % m.objVal)
print('SOLUTION: 矩阵解为')
print "   ",
for w in warehouses:
    print "[B{0}]".format(w),
print 
for p in plants:
    print "[A{0}]".format(p), 
    if open[p] > 0.99:
        # print('Plant %s open' % p)
        for w in warehouses:
            #if transport[w,p].x > 0:
            #    print('  Transport %g units to warehouse %s' % \
            #          (transport[w,p].x, w))
            print transport[p,w].x, 
        print
    else:
        print('Plant %s closed!' % p)

# print transport[0,1].x
# print transport[1,0].x
# print transport[3,0].x
print "\n最优值为：", m.ObjVal


TOTAL COSTS: 28
SOLUTION: 矩阵解为
    [B0] [B1] [B2] [B3]
[A0] 0.0 -0.0 -0.0 1.0
[A1] -0.0 1.0 -0.0 -0.0
[A2] 1.0 -0.0 -0.0 -0.0
[A3] -0.0 0.0 1.0 -0.0

最优值为： 28.0


```
# 解出的最优解满足生产运输约束, 与 p.128  最优解为  28(小时) 答案一致
```